In [ ]:
from valurap2 import gcode
from valurap2 import path_planning
from valurap2 import emulate

from ipywidgets import widgets
from ipywidgets import Layout

import imp
imp.reload(gcode)
imp.reload(path_planning)

pp = path_planning.PathPlanner()
pp.emu_in_loop = True

if 1:
    speed_k = 1.0
    pp.delta_e_err = 50
    pp.delta_ve_err = 20
    pp.max_ea = 10000
    pp.accel_step = 5000
else:
    pp.delta_e_err = 100
    pp.delta_ve_err = 100
    pp.max_ea = 20000
    pp.accel_step = 5000
    speed_k = 10.0

all_deltas = []

text = widgets.Textarea(layout=Layout(width='auto', height='450px'))
display(text)

text_val = ""
total_deltas = {}

if 1:
    #fn = "simple-test.gcode"
    #fn = "box20.gcode"
    fn = "box20_void_dual.gcode"
    #fn = "chess5.gcode"
    pp.gen_layers(fn, speed_k=speed_k, max_layers=300)
else:
    #sg = gcode.file_reader("helmet.gcode")
    sg = gcode.file_reader("test18.gcode")
    for s in sg:
        if isinstance(s, gcode.do_segment):
            hdr = "############# L{} - L{} ###############".format(s.path[0][4], s.path[-1][4])
            print("\n\n{}\n".format(hdr))
            text.value = hdr + "\n...\n...\n" + text_val

            print("path len:", len(s.path), s.path[0:2])
            try:
                path, slowdowns = pp.make_path(s.path, speed_k=speed_k)
                if path is None:
                    continue
                #print(path)
                #print(slowdowns)
                slowdowns, updated, cc = pp.process_corner_errors(path, slowdowns)
                slowdowns, updated = pp.reverse_pass(path, slowdowns)
                slowdowns, updated = pp.forward_pass(path, slowdowns)
                df, profile = pp.gen_segments_float(path, slowdowns)
                print(pp.max_deltas)
                md_str = []
                td_str = []
                for k,v in pp.max_deltas.items():
                    md_str.append("{}: {:10.3f}".format(k, v))
                    total_deltas[k] = max(total_deltas.get(k, 0), v)
                    td_str.append("{}: {:10.3f}".format(k, total_deltas[k]))
                text.value = (hdr + "\n" + " ".join(md_str) + "\n" + " ".join(td_str) + "\n" + text_val)[:10000]
                text_val = text.value

                all_deltas.append(pp.max_deltas.copy())
            except Exception:
                pp.print_msg_buf()
                raise

In [ ]:
import os.path
import pickle

#base_fn = "align3"
#base_fn = "box20"
base_fn = "box20_void_dual"
#base_fn = "chess5"
layer = 0
tupled_segment = None
while True:
    fn = "{}_{:05d}.layer".format(base_fn, layer)
    if not os.path.exists(fn):
        break
    print(fn)
    data = pickle.load(open(fn, "rb"))
    for d in data:
        if d[0] == "segment":
            print(d[0], len(d), len(d[1]), len(d[2]))
            for p in d[2][:5]:
                print("   ", p)
            tupled_segment = d[2]
            break
        elif d[0] == "start":
            print("   ", d)
        elif d[0] == "end_state":
            print("   ", d)
        elif d[0] == "extruder_switch":
            print("   extruder_switch", d[1])
        else:
            print(d[0], len(d))
    layer += 1
    if layer > 10:
        break

In [ ]:
from valurap2.profile import ProfileSegment
from valurap2 import emulate

#print(tupled_segment)
pr_opt = []
for dt, segs in tupled_segment:
    pr_opt.append([
        dt, [ProfileSegment.from_tuple(s) for s in segs]
    ])
steps = emulate.emulate(pr_opt, apg_states=pp.apg_states, no_tracking=False)
display(steps)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
#steps.plot(x="t", y=["apg5_v"], style="*")
steps.plot(x="apg0_x", y=["apg2_x"], style="*")

In [ ]:
from valurap2 import buf_commands
import pickle

imp.reload(buf_commands)
cb = buf_commands.CommandBuffer(debug=True)
cb.add_segments_head(pp)
cb.add_segments(pr_opt)
cb.add_segments_foot()
#print(list(cb.buffer))
pickle.dump(list(cb.buffer), open("commands.pck", "wb"))

In [ ]:
import pandas
import numpy
import imp
from valurap2 import path_planning

imp.reload(path_planning)
pp = path_planning.PathPlanner()
pp.spms = pp.print_spms
pp.init_apgs()
print(pp.apg_states)

pp.ext_to_code([100], axes=["X1"])

In [ ]:
from valurap2 import rest_client
import time

rc = rest_client.Client()
rc.home()
rc.wait_idle()
if 1:
    rc.move(E1=-30, E2=-30)
    rc.wait_idle()
    rc.moveto(X1=0, X2=165, Y=0)
    rc.wait_idle()
    rc.moveto(Z=9.60)
    rc.wait_idle()
    rc.move(E1=-0.1, E2=-0.1)
    rc.wait_idle()
    rc.exec_binary(cb.buffer)
    for i in range(10):
        print(rc.state())
        time.sleep(1)
    rc.wait_idle()
    rc.move(Z=50)
    rc.wait_idle()
    rc.moveto(Y=-200)
    rc.wait_idle()
    
print("done")


In [ ]:
#rc.moveto(Y=0, X1=-30, X2=165)
rc.move(Z=-2)
rc.wait_idle()


In [ ]:
from valurap2 import rest_client

rc = rest_client.Client()
rc.home()
rc.wait_idle()

rc.move(Z=30)
rc.wait_idle()

rc.moveto(X1=0, Y=0)
rc.wait_idle()

rc.move(Z=-12)
rc.wait_idle()

rc.abort()

In [ ]:
#rc.move(Y=100)
#rc.move(Y=-56.8)
#rc.move(Y=-0.05)
#rc.move(X2=105)
rc.move(X2=-300)
#rc.move(BLZ=0.5, FLZ=0.5)
rc.wait_idle()


In [ ]:
rc.exec_binary(cb.buffer)

In [ ]:
rc.abort()

In [ ]:
rc.home()
rc.wait_idle()
rc.abort()

In [ ]:
rc.state()

In [ ]:
rc.move(E1=-100, E2=-100)
rc.wait_idle()

In [ ]:
import os.path
import pickle
from valurap2.profile import ProfileSegment
from valurap2 import emulate
from valurap2 import rest_client
from valurap2 import buf_commands
import time

e1_x_offset = 0.0
e1_y_offset = 0.0
e1_z_offset = 9.3   #+ 20

e2_x_offset = 7.29 - 0.5
e2_y_offset = 122.6
e2_z_offset = 12.46 #+ 20

x1_parking = -190
x2_parking = 170
y_parking = -200

#base_fn = "box20"
base_fn = "box20_void_dual"

rc = rest_client.Client()
rc.home()
rc.wait_idle()
rc.move(E1=-30, E2=-30)
rc.wait_idle()
rc.moveto(X1=x1_parking, X2=x2_parking, Y=y_parking, mode="print,e1,e2")
rc.wait_idle()
#rc.moveto(Z=9.60, mode="print,e1,e2")
#rc.wait_idle()
    
def send_segment(segment, accel_step, rc):
    #print(tupled_segment)
    pr_opt = []
    for dt, segs in segment:
        pr_opt.append([
            dt, [ProfileSegment.from_tuple(s) for s in segs]
        ])
        
    cb = buf_commands.CommandBuffer(debug=False)
    cb.add_segments_head(accel_step=accel_step)
    cb.add_segments(pr_opt)
    cb.add_segments_tail()
    state = rc.state()
    while state["buf_len"] > 1000000:
        time.sleep(1)
        state = rc.state()
        #print(state)
    
    rc.exec_binary(cb.buffer)


layer = 1
current_extruder = "E1"

target_z = 0

while True:
    fn = "{}_{:05d}.layer".format(base_fn, layer)
    if not os.path.exists(fn):
        break
    print(fn)
    data = pickle.load(open(fn, "rb"))
    for d in data:
        if d[0] == "segment":
            print(d[0], len(d), len(d[1]), len(d[2]), d[1])
            for p in d[2][:5]:
                print("   ", p)
            tupled_segment = d[2]
            if "accel_step" not in d[1]:
                d[1]["accel_step"] = d[1]["acc_step"]
            send_segment(tupled_segment, d[1]["accel_step"], rc)
        elif d[0] == "start":
            print("   ", d)
            code = d[5]
            target = d[1]
            target_z = target["Z"]
            if code:
                print("code", list(code.keys()), len(code["segment"]), code["segment"][:5])
                send_segment(code["segment"], code["accel_step"], rc)
            else:
                print("no code, target", target)
                state = rc.state()
                while not state["idle"]:
                    time.sleep(1)
                    state = rc.state()
                    #print(state)
                if current_extruder == "E1":
                    rc.moveto(Z=target["Z"] + e1_z_offset, mode="print,e1,e2")
                    rc.wait_idle()
                    rc.moveto(X1=target["X"] + e1_x_offset, Y=target["Y"] + e1_y_offset, mode="print,e1,e2")
                    rc.wait_idle()
                else:
                    rc.moveto(Z=target["Z"]  + e2_z_offset, mode="print,e1,e2")
                    rc.wait_idle()
                    rc.moveto(X2=target["X"] + e2_x_offset, Y=target["Y"] + e2_y_offset, mode="print,e1,e2")
                    rc.wait_idle()
        elif d[0] == "end_state":
            print("   ", d)
        elif d[0] == "do_home":
            print("   ", d)
        elif d[0] == "extruder_switch":
            new_extruder = "E{}".format(d[1] + 1)
            if current_extruder != new_extruder:
                state = rc.state()
                while not state["idle"]:
                    time.sleep(0.1)
                    state = rc.state()
                    #print(state)
                rc.move(Z=5, mode="print,e1,e2")
                rc.wait_idle()
                if current_extruder == "E1":
                    rc.moveto(X1=x1_parking, mode="print,e1,e2")
                    rc.wait_idle()
                else:
                    rc.moveto(X2=x2_parking, mode="print,e1,e2")
                    rc.wait_idle()
            current_extruder = new_extruder
        else:
            print(d[0], len(d))
    layer += 1
    if target_z > 1.0:
        break

state = rc.state()
while not state["idle"]:
    time.sleep(10)
    state = rc.state()
    print(state)
        
rc.wait_idle()
rc.move(Z=50)
rc.wait_idle()
rc.moveto(X1=x1_parking, X2=x2_parking, Y=y_parking, mode="print,e1,e2")
rc.wait_idle()
    
print("done")
